In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import os
import s3fs
import matplotlib.pyplot as plt 
import xgboost as xgb

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.regularizers import l2

from modules.utils import pre_process_data, encoded_categorical_features

In [12]:
X_test = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_test_Hi5.csv", sep=",")

/var/folders/41/lv7xy5r91x7ckkgx5zqmgpxh0000gn/T/ipykernel_10906/182843433.py:1: DtypeWarning: Columns (1,5,108,110,111,115,117,118,122,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  X_test = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/X_test_Hi5.csv", sep=",")


In [13]:
X_test_copy = X_test


In [ ]:
# Convert to datetime
X_test["piezo_measurement_date"] = pd.to_datetime(X_test["piezo_measurement_date"], errors="coerce")

In [ ]:
print(len(X_test.columns))

In [14]:
X_test["piezo_groundwater_level_category"] = 0

x_test_without_nan, y_train, y_origginal = pre_process_data(X_test)

/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  retur

['prelev_volume_0', 'prelev_usage_label_0', 'prelev_volume_obtention_mode_label_0', 'prelev_volume_1', 'prelev_usage_label_1', 'prelev_volume_obtention_mode_label_1', 'prelev_volume_2', 'prelev_usage_label_2', 'prelev_volume_obtention_mode_label_2', 'prelev_other_volume_sum']


/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/elouan/anaconda3/envs/challenge_ENS_env_QRT_2024/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  retur

In [ ]:
print(len(x_test_without_nan.columns))

In [15]:
x_test_without_nan["insee_%_agri"] = x_test_without_nan["insee_%_agri"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)
x_test_without_nan["insee_med_living_level"] = x_test_without_nan["insee_med_living_level"].replace(
    {'N/A - résultat non disponible': 0}  # Replace with 0 or any value you choose
).astype(float)
x_test_without_nan["insee_%_ind"] = x_test_without_nan["insee_%_ind"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)
x_test_without_nan["insee_%_const"] = x_test_without_nan["insee_%_const"].replace(
    {'N/A - division par 0': 0}  # Replace with 0 or any value you choose
).astype(float)

In [16]:
X_final = encoded_categorical_features(x_test_without_nan)

In [17]:
X_final = pd.DataFrame(X_final)
X_final = X_final.fillna(X_final.mean())

In [ ]:


# Create an XGBClassifier instance
#loaded_model = xgb.XGBClassifier()

# Load the model from a file
#loaded_model.load_model("models/xgboost_model_2.json")

In [81]:
from tensorflow.keras.models import load_model

model = load_model('../models/ML_model_2.keras')

In [82]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 256)            │        22,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 191,249 (747.07 KB)

 Trainable params: 63,749 (249.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 127,500 (498.05 KB)

In [83]:
print(X_final.shape)


(611208, 86)


In [84]:
y_pred = model.predict(X_final)

19101/19101 ━━━━━━━━━━━━━━━━━━━━ 26s 1ms/step


In [85]:
y_pred = pd.DataFrame(y_pred)
y_pred.head()

,0,1,2,3,4
0,0.514418,0.048854,4.336909e-01,3.023097e-03,1.395575e-05
1,0.426365,0.278095,2.906567e-01,4.803240e-03,8.035012e-05
2,0.308070,0.038418,6.531686e-01,8.192209e-05,2.615549e-04
3,0.000012,0.302740,8.134562e-08,6.972478e-01,3.595743e-16
4,0.066004,0.000811,9.075943e-01,8.465538e-08,2.559146e-02


In [86]:
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert to class labels
y_pred_classes = pd.DataFrame(y_pred_classes)
y_pred_classes.head()

,0
0,0
1,0
2,2
3,3
4,2


In [87]:
# Convert predictions to DataFrame


# Add a row index starting from 2331796
y_pred_classes["row_index"] = X_test_copy["row_index"]
# Define category mapping
category_mapping = {
    4: "Very Low",
    2: "Low",
    0: "Average",
    1: "High",
    3: "Very High"
}

# Map the category values
y_pred_classes["piezo_groundwater_level_category"] = y_pred_classes[0].map(category_mapping)

# Display the final DataFrame
y_pred_classes = y_pred_classes.drop(columns=[0])

In [ ]:
X_final.info()

In [ ]:
X_test_copy["row_index"]

In [60]:
y_pred_classes

,row_index,piezo_groundwater_level_category
0,2331795,Average
1,2331796,High
2,2331797,Low
3,2331798,High
4,2331799,Average
...,...,...
611203,3610818,Average
611204,3610819,Average
611205,3610820,Average
611206,3610821,Average


In [88]:
y_pred_classes.to_csv("/Users/elouan/Data/Data_Hickathon5/Submissions/y_ML_final_2.csv", index=False)

In [35]:
y_example = pd.read_csv("/Users/elouan/Data/Data_Hickathon5/Submissions/y_test_submission_example_Hi5.csv", sep=",")

In [36]:
y_example

,row_index,piezo_groundwater_level_category
0,2331795,My prediction
1,2331796,My prediction
2,2331797,My prediction
3,2331798,My prediction
4,2331799,My prediction
...,...,...
611203,3610818,My prediction
611204,3610819,My prediction
611205,3610820,My prediction
611206,3610821,My prediction
